# Part 3: Data Analytics

In [7]:
import os, json
import pandas as pd
import fsspec
import boto3

session = boto3.Session(profile_name="rearc")
s3 = session.client("s3")

bucket = "rearc-quest-tim"
csv_key = "pr.data.0.Current"
resp = s3.list_objects_v2(Bucket=bucket, Prefix="datausa/")
latest = max(resp["Contents"], key=lambda o: o["LastModified"])
json_key = latest["Key"]

print(f"Using latest population file: {json_key}")

Using latest population file: datausa/2025-07-01.json


In [8]:
bls = (
    pd.read_csv(f"s3://{bucket}/{csv_key}", sep="\t")
      .rename(columns=lambda c: c.strip())
      .assign(series_id=lambda df: df["series_id"].str.strip())
      .query("period.str.startswith('Q')", engine="python")
      .astype({"year": int, "value": float})
)

with fsspec.open(f"s3://{bucket}/{json_key}") as f:
    data = json.load(f)

pop = (
    pd.json_normalize(data["data"])[["Year", "Population"]]
      .rename(columns={"Year": "year", "Population": "population"})
      .astype({"year": int})
)

In [3]:
mask = pop["year"].between(2013, 2018)
mean_p = pop.loc[mask, "population"].mean()
std_p  = pop.loc[mask, "population"].std()

print(f"Mean population 2013-2018: {mean_p:,.0f}")
print(f"Std-dev population 2013-2018: {std_p:,.0f}")

Mean population 2013-2018: 317,437,383
Std-dev population 2013-2018: 4,257,090


In [9]:
### JUST TO TEST THE INPUT TABLE

# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.width", None) 

# bls.head(20)

In [10]:
best_year = (
    bls.groupby(["series_id", "year"], as_index=False)
       .agg(year_sum=("value", lambda s: round(s.sum(), 1)))
       .sort_values(["series_id", "year_sum"], ascending=[True, False])
       .drop_duplicates("series_id", keep="first")
       .reset_index(drop=True)
)

best_year.head()

,series_id,year,year_sum
0,PRS30006011,2022,20.5
1,PRS30006012,2022,17.1
2,PRS30006013,1998,705.9
3,PRS30006021,2010,17.7
4,PRS30006022,2010,12.4


In [11]:
target = (
    bls.query("series_id == 'PRS30006032' and period == 'Q01'")
        .loc[:, ["series_id", "year", "period", "value"]]
)

joined = (
    target.merge(pop, on="year", how="left")
          .dropna(subset=["population"])
          .rename(columns={"population": "Population"})
          .astype({"Population": "int64"})
          .sort_values("year")
          .reset_index(drop=True)
)

joined.head()

,series_id,year,period,value,Population
0,PRS30006032,2013,Q01,0.5,311536594
1,PRS30006032,2014,Q01,-0.1,314107084
2,PRS30006032,2015,Q01,-1.7,316515021
3,PRS30006032,2016,Q01,-1.4,318558162
4,PRS30006032,2017,Q01,0.9,321004407
